In [2]:
from azureml.core import Workspace, Experiment

# ws = Workspace.get(name="udacity-project") Must be updated with values from the workspace
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

NameError: name 'ws' is not defined

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
from azureml.core.compute_target import ComputeTargetException
cluster_name = "azuremlCluster"
try:
    compute_cluster = ComputeTarget(workspace=ws, name=cluster_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

compute_cluster.wait_for_completion(show_output=True)

In [ ]:
from azureml.core import ScriptRunConfig
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
from azureml.core import Environment

# Specify an sklearn - enabled environment
sklearn_env = Environment.get(workspace=ws, name="AzureML-AutoML")

# Specify parameter sampler
ps = RandomParameterSampling(
    {'C': uniform(0.001, 4.0),
    'max_iter': choice([1000, 1500, 2000, 2500, 5000])}
)

# Specify a Policy
policy = BanditPolicy(slack_factor=0.1, evaluation_interval=2, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
#est = SKLearn(source_directory=".", entry_script='train.py', compute_target=compute_cluster)
est = ScriptRunConfig(source_directory='.', script='train.py', 
    compute_target=compute_cluster,
    environment=sklearn_env   
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    run_config=est,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=10
)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
experiment = Experiment(ws, "HyperdriveExperiment")
hyperdrive_run = experiment.submit(config=hyperdrive_config)
hyperdrive_run.wait_for_completion(show_output=True)

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
from azureml.widgets import RunDetails
best_run = hyperdrive_run.get_best_run_by_primary_metric()
RunDetails(hyperdrive_run).show()

# joblib.dump(best_run, 'model.joblib') # didn't end up working
model = best_run.register_model(model_name='hyperdrive_model', model_path='outputs/model.joblib')

#best_run_metrics = best_run.get_metrics()
#parameter_values = best_run.get_details()['runDefinition']['arguments']

#print('Best Run Id: ', best_run.id)
#print('\n Accuracy:', best_run_metrics['accuracy'])




In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")


In [ ]:
from train import clean_data
from azureml.core.dataset import Dataset

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x_concat = pd.concat([x, y], axis=1)

# Register x dataframe as a dataset to be consumed by automl
datastore = ws.get_default_datastore()
TabularDatasetFactory.register_pandas_dataframe(
    dataframe=x_concat,
    target=datastore,
    name="x_tabular_dataset"
)
x_tabular_dataset = Dataset.get_by_name(name='x_tabular_dataset',
    workspace=ws)

In [3]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=x_tabular_dataset,
    label_column_name="y",
    n_cross_validations=5,
    compute_target=compute_cluster
)

ModuleNotFoundError: No module named 'azureml.train'

In [2]:
# Submit your automl run

### YOUR CODE HERE ###
automl_experiment = Experiment(ws, 'automl_experiment')
run = automl_experiment.submit(config=automl_config)
run.wait_for_completion(show_output=True)

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run, model = run.get_output()
joblib.dump(model, 'best_automl_model.joblib')